In [1]:
import datetime as dt
import json
import re
import time

import datazimmer as dz
import pandas as pd

from aswan import get_soup, get_json, add_url_params, ConnectionError
from tqdm import tqdm

In [2]:
fifa_rank_base = dz.SourceUrl("https://www.fifa.com/fifa-world-ranking/men")

2023-06-28 20:06.50 [warning  ] can't get module name, likely due to notebook call


In [3]:
soup = get_soup(fifa_rank_base)

In [4]:
dr = '"dates"\:\[.*?\]'

In [5]:
date_ids = json.loads("{" + re.findall(dr, str(soup))[0] + "}")

In [6]:
ranks_by_date = {}
for date_dic in tqdm(date_ids["dates"]):
    did = date_dic["id"]
    ranks_by_date[did] = get_json(
        "https://www.fifa.com/api/ranking-overview",
        params={"locale": "en", "dateId": date_dic["id"]},
    )
    time.sleep(0.1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 324/324 [01:34<00:00,  3.44it/s]


In [29]:
uefa_seasons = list(range(1998, dt.date.today().year + 1))

In [31]:
outer_keys = ["countryName", "displayName"]

all_ranks = []
for season_year in tqdm(uefa_seasons):
    for pref in [
        "coefficientType=MEN_CLUB&coefficientRange=OVERALL",
        "coefficientType=MEN_ASSOCIATION&coefficientRange=SEASONAL",
        "coefficientType=MEN_ASSOCIATION&coefficientRange=OVERALL",
    ]:
        url = (
            f"https://comp.uefa.com/v2/coefficients?{pref}&"
            f"seasonYear={season_year}&page=1&pagesize=500&language=en"
        )
        if (season_year < 2005) and ("CLUB" in pref):
            continue

        club_json = get_json(url)
        for member in club_json["data"]["members"]:
            outer_d = {k: member["member"][k] for k in outer_keys}
            for ranks in member["seasonRankings"]:
                all_ranks.append(
                    outer_d
                    | ranks
                    | {"collection_range": pref.split("=")[-1], "collection_year": season_year}
                )
    time.sleep(1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:44<00:00,  1.71s/it]


In [32]:
pd.DataFrame(all_ranks).drop_duplicates().to_csv(dz.get_raw_data_path("uefa-ranks.csv"), index=None)

In [33]:
date_dic = {d["id"]: d["text"] for d in date_ids["dates"]}

In [34]:
todrop = ["flag", "tag"]

In [35]:
def flat(d0):
    d = d0.copy()
    ri = d.pop("rankingItem")
    return {k: v for k, v in (ri | d).items() if k not in todrop}

In [36]:
fifa_dfs = []
for k, v in ranks_by_date.items():
    fifa_dfs.append(pd.DataFrame(map(flat, v["rankings"] or [])).assign(date=date_dic[k]))

In [37]:
pd.concat(fifa_dfs, ignore_index=True).drop_duplicates().to_csv(
    dz.get_raw_data_path("fifa-ranks.csv"), index=None
)